# Practical 4: Particle Swarm Optimisation for Feature Selection

In today's lab, we will learn how to perform feature selection using particle swarm optimistation, utilising the __[pyswarms](https://pyswarms.readthedocs.io/en/latest/index.html)__ toolkit.


The aim of feature selection is to automatically select suitable data features suitable for use in AI models and algorithms. This is an important process before model training (covered in machine learning and deep learning parts of the course) as too many or redundant features can negatively impact the learning and accuracy of the model.

<hr style="border:1px solid black"> </hr>

## Step 1: Install and importing relevant modules 

In [1]:
pip install pyswarms

     |████████████████████████████████| 104 kB 3.6 MB/s eta 0:00:01
     |████████████████████████████████| 829 kB 5.7 MB/s eta 0:00:01
  Using cached PyYAML-6.0-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (701 kB)
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491058 sha256=1b883fcd5e71d697ed3fcbc55bffa456c3901b33c35ec6a87c77db76e5bf6a83
  Stored in directory: /home/changhyun/.cache/pip/wheels/8e/70/28/3d6ccd6e315f65f245da085482a2e1c7d14b90b30f239e2cf4
Successfully built future
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import time
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

import pyswarms as ps

<hr style="border:1px solid black"> </hr>

## Step 2: Create dataset with redundant features

Use sklearn's `make_classification` function to create a dataset with a large number of redundant features

In [101]:
# Define the number of features for each type
n_features = 200
n_informative = 20
n_redundant = 100
n_repeated = 50
n_useless = 30

# Create Labels
informative_labels = [f'Informative {ii}' for ii in range(1, n_informative + 1)]
redundant_labels = [f'Redundant {ii}' for ii in range(n_informative + 1, n_informative + n_redundant + 1)]
repeated_labels = [f'Repeated {ii}' for ii in range(n_informative + n_redundant+ 1, n_informative + n_redundant + n_repeated + 1)]
useless_labels = [f'Useless {ii}' for ii in range(n_informative + n_redundant + n_repeated + 1, n_features + 1)]
labels = informative_labels + redundant_labels + repeated_labels + useless_labels

# Get data
X, y = make_classification(n_samples = 5000, n_features = n_features,
                           n_informative = n_informative,
                           n_redundant = n_redundant , n_repeated = n_repeated,
                           n_clusters_per_class = 2, class_sep = 0.5, flip_y = 0.05,
                           random_state = 42, shuffle = False)


In [37]:
labels

['Informative 1',
 'Informative 2',
 'Informative 3',
 'Informative 4',
 'Informative 5',
 'Informative 6',
 'Informative 7',
 'Informative 8',
 'Informative 9',
 'Informative 10',
 'Informative 11',
 'Informative 12',
 'Informative 13',
 'Informative 14',
 'Informative 15',
 'Informative 16',
 'Informative 17',
 'Informative 18',
 'Informative 19',
 'Informative 20',
 'Redundant 21',
 'Redundant 22',
 'Redundant 23',
 'Redundant 24',
 'Redundant 25',
 'Redundant 26',
 'Redundant 27',
 'Redundant 28',
 'Redundant 29',
 'Redundant 30',
 'Redundant 31',
 'Redundant 32',
 'Redundant 33',
 'Redundant 34',
 'Redundant 35',
 'Redundant 36',
 'Redundant 37',
 'Redundant 38',
 'Redundant 39',
 'Redundant 40',
 'Redundant 41',
 'Redundant 42',
 'Redundant 43',
 'Redundant 44',
 'Redundant 45',
 'Redundant 46',
 'Redundant 47',
 'Redundant 48',
 'Redundant 49',
 'Redundant 50',
 'Redundant 51',
 'Redundant 52',
 'Redundant 53',
 'Redundant 54',
 'Redundant 55',
 'Redundant 56',
 'Redundant 57',


In role of the code in the next cell block is to standardise the data

**Question** What does this do to the data, and why might this be important for feature selection?

It is to scale data into a boundary that prevents extremely large or small values to be dominant during model training

In [102]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X)
X = scaler.transform(X)

<hr style="border:1px solid black"> </hr>

### Step 3: Set-up the objective function

We’ll be using the optimizer `pyswarms.discrete.BinaryPSO` to perform feature subset selection. 

For a Binary PSO, the position of the particles are expressed in two terms: 1 or 0 (or on and off). Mathematically this is defined as: 
                    $x=[x_{1},x_{2},x_{3},…,x_{d}]$   where   $x_{i}\in {0,1}$

The objective function we will be using is taken from this paper and is define as:

$f(X)=\alpha(1−P)+(1−\alpha)(1− \dfrac{N_{f}}{N_{t}})$

Where $\alpha$ is a hyperparameter tradeoffs the performance of classifier $P$, with the ratio of the size of the feature subset $N_{f}$ with respect to the total number of features $N_{t}$.

**First** we need to define a function which calcualtes the objective function per particle. 

It return the object loss score for a *single* particle

 Do you understand the purpose of line in the function? 

In [103]:
"""Computes for the objective function per particle

Inputs
------
m : numpy.ndarray
    Binary mask that can be obtained from BinaryPSO, will
    be used to mask features.
alpha: float (default is 0.5)
    Constant weight for trading-off classifier performance
    and number of features

Returns
-------
numpy.ndarray
    Computed objective function
"""

# Create an instance of the classifier
classifier = RandomForestClassifier(max_depth=2,n_estimators=10)

# Define objective function
def f_per_particle(m, alpha):

    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    
    # Get the subset of the features from the binary mask
    if np.count_nonzero(m) == 0:
        X_train_subset = X_train
        X_test_subset = X_test
    else:
        X_train_subset = X_train[:,m==1]
        X_test_subset = X_test[:,m==1]
        
    # Perform classification and store performance in P
    classifier.fit(X_train_subset, y_train)
    P = (classifier.predict(X_test_subset) == y_test).mean()

    # Compute for the objective function
    j = (alpha * (1.0 - P)
        + (1.0 - alpha) * (1 - (X_train_subset.shape[1] /X_train.shape[1])))
    
    return j

**Next** we define the higher level objective function to be evaluated in pyswarms built in optimiser. 

It returns the object loss score for *all* particles

In [104]:
"""Higher-level method to do classification in the
whole swarm.

Inputs
------
M: numpy.ndarray of shape (n_particles, dimensions)
    The swarm that will perform the search

Returns
-------
numpy.ndarray of shape (n_particles, )
    The computed loss for each particle
"""

def f(M, alpha=0.99):

    n_particles = M.shape[0]
    
    j = [f_per_particle(M[i], alpha) for i in range(n_particles)]

    return np.array(j)

<hr style="border:1px solid black"> </hr>

## Step 3: Set-up the optimisation parameters and run the selections algorithm

In the `options` dictionary below: 
- `k`represents the neighbors to be considered when calculating the best known position of the swarm
- `p`represents a distance metric used in the optimisation algorithm. 

**Question** What are the purpose of the other three parameters? 

In [107]:
# Initialize swarm, arbitrary
options = {'c1': 1.5, 'c2': 1.5, 'w':0.5, 'k': 4, 'p':2}

# Call instance of PSO
dimensions = X.shape[1] # dimensions should be the number of features

optimizer = ps.discrete.BinaryPSO(n_particles=60, dimensions=dimensions, options=options)

# Perform optimization
tic = time.perf_counter()
cost, pos = optimizer.optimize(f, iters=20, verbose=2)
toc = time.perf_counter()
print(f"Optimiser ran for {toc - tic:0.4f} seconds")
selected = np.count_nonzero(pos)
print("Optimiser retained " + str(selected)   + " features")

2022-05-12 16:38:55,505 - pyswarms.discrete.binary - INFO - Optimize for 20 iters with {'c1': 1.5, 'c2': 1.5, 'w': 0.5, 'k': 4, 'p': 2}
pyswarms.discrete.binary: 100%|██████████████████████████|20/20, best_cost=0.264
2022-05-12 16:40:05,246 - pyswarms.discrete.binary - INFO - Optimization finished | best cost: 0.26430000000000003, best pos: [1 0 0 1 0 1 0 0 1 1 0 1 0 1 1 0 0 0 1 0 1 0 1 1 1 1 0 0 1 1 1 0 1 0 1 0 1
 0 0 1 0 1 0 0 0 1 0 1 1 1 1 0 0 0 0 1 1 0 1 0 0 1 1 0 1 0 0 0 1 0 1 0 0 1
 0 0 1 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 0 1 1 0 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1
 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 0 1 1 0 1 1 0 1 0 1 1 0 0
 1 0 1 0 0 0 1 0 1 1 1 1 1 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 0 1 1 0 1 1 0 1 1
 0 0 0 0 1 0 0 0 0 1 1 0 1 1 0]


Optimiser ran for 69.7429 seconds
Optimiser retained 110 features


<hr style="border:1px solid black"> </hr>

## Step 4: Evalute the performamce of the selected features

We evaluate the effectiveness of the selected features, we can use two Random Forest Classifiers, one trained on the full feature set, the other trained on the selected feature sets, and compare the outputs

In [113]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

c1 = RandomForestClassifier(max_depth=2, n_estimators=10, random_state=42)
c1.fit(X_train, y_train)
full_performance = (c1.predict(X_test) == y_test).mean()
print('Full Feature set performance: %.3f' % (full_performance))

c2 = RandomForestClassifier(max_depth=2, n_estimators=10, random_state=42)
c2.fit(X_train[:,pos==1], y_train)
subset_performance = (c2.predict(X_test[:,pos==1]) == y_test).mean()
print('Subset Feature set performance: %.3f' % (subset_performance))

Full Feature set performance: 0.650
Subset Feature set performance: 0.677


<hr style="border:1px solid black"> </hr>

## Exercise 1

Observe how altering key parameters changes the performance and running time of the optimiser.

Parameters to consider altering include
- alpha
- Swarm options
- Number of particles
- Number of optimiser iterations

In [48]:
# Alpha is to control weight between model performances and num of features selected (previous: 0.5)
# c1 -> cognitive param, c2 -> social param, w -> inertia, k -> num of nearest particles, p -> distance
alpha = 0.1
options = {'c1': 1.1, 'c2': 1.1, 'w':0.9, 'k': 2, 'p':2}

optimizer = ps.discrete.BinaryPSO(n_particles=100, dimensions=dimensions, options=options)

# Perform optimization
tic = time.perf_counter()
cost, pos = optimizer.optimize(f, iters=10, verbose=2)
toc = time.perf_counter()
print(f"Optimiser ran for {toc - tic:0.4f} seconds")
selected = np.count_nonzero(pos)
print("Optimiser retained " + str(selected)   + " features")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

c1 = RandomForestClassifier(max_depth=2, n_estimators=10, random_state=42)
c1.fit(X_train, y_train)
full_performance = (c1.predict(X_test) == y_test).mean()
print('Full Feature set performance: %.3f' % (full_performance))

c2 = RandomForestClassifier(max_depth=2, n_estimators=10, random_state=42)
c2.fit(X_train[:,pos==1], y_train)
subset_performance = (c2.predict(X_test[:,pos==1]) == y_test).mean()
print('Subset Feature set performance: %.3f' % (subset_performance))

2022-05-12 15:34:57,859 - pyswarms.discrete.binary - INFO - Optimize for 10 iters with {'c1': 1.1, 'c2': 1.1, 'w': 0.9, 'k': 2, 'p': 2}
pyswarms.discrete.binary: 100%|██████████████████████████|10/10, best_cost=0.313
2022-05-12 15:35:58,640 - pyswarms.discrete.binary - INFO - Optimization finished | best cost: 0.3134848484848485, best pos: [1 0 1 0 0 1 0 1 1 1 1 1 0 1 1 0 1 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1
 0 0 0 0 1 0 1 0 1 1 1 0 1 1 1 1 0 0 1 0 1 1 1 0 0 1 1 0 1 1 0 1 0 0 0 1 0
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 0 1 0 1
 1 0 1 1 0 1 0 0 0 0 1 1 0 0 0 1 1 1 1 1 0 0 1 1 1 1 1 0 0 1 1 0 0 1 0 1 1
 1 1 1 0 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 0 1 1 1 0 0 1 1 0 0 0 1 1 1
 0 1 0 1 0 1 0 0 1 0 0 1 1 0 0]


Optimiser ran for 60.7822 seconds
Optimiser retained 130 features
Full Feature set performance: 0.650
Subset Feature set performance: 0.665


<hr style="border:1px solid black"> </hr>

## Exercise 2

Implement feature selection on the *Breast Cancer Wisconsin Diagnostic Dataset*. The features present in this dataset are computed from a digitized image of a fine needle aspirate (FNA) of a breast mass. They describe characteristics of the cell nuclei present in the image. The features are labelled as coming from either a malignant or benign sample.

The data can be imported directly from sklearn

In [54]:
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer()
print(data.DESCR) 

X, y = data.data, data.target

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

In [56]:
scaler = preprocessing.StandardScaler()
X = scaler.fit_transform(X)
X

array([[ 1.09706398, -2.07333501,  1.26993369, ...,  2.29607613,
         2.75062224,  1.93701461],
       [ 1.82982061, -0.35363241,  1.68595471, ...,  1.0870843 ,
        -0.24388967,  0.28118999],
       [ 1.57988811,  0.45618695,  1.56650313, ...,  1.95500035,
         1.152255  ,  0.20139121],
       ...,
       [ 0.70228425,  2.0455738 ,  0.67267578, ...,  0.41406869,
        -1.10454895, -0.31840916],
       [ 1.83834103,  2.33645719,  1.98252415, ...,  2.28998549,
         1.91908301,  2.21963528],
       [-1.80840125,  1.22179204, -1.81438851, ..., -1.74506282,
        -0.04813821, -0.75120669]])

<hr style="border:1px solid black"> </hr>

In [95]:
def f(M, alpha=1):

    n_particles = M.shape[0]
    
    j = [f_per_particle(M[i], alpha) for i in range(n_particles)]

    return np.array(j)

In [98]:
options = {'c1': 1.9, 'c2': 1.9, 'w':0.5, 'k': 10, 'p':2}

# Call instance of PSO
dimensions = X.shape[1] # dimensions should be the number of features

optimizer = ps.discrete.BinaryPSO(n_particles=200, dimensions=dimensions, options=options)

# Perform optimization
tic = time.perf_counter()
cost, pos = optimizer.optimize(f, iters=10, verbose=2)
toc = time.perf_counter()
print(f"Optimiser ran for {toc - tic:0.4f} seconds")
selected = np.count_nonzero(pos)
print("Optimiser retained " + str(selected)   + " features")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

c1 = RandomForestClassifier(max_depth=2, n_estimators=10, random_state=42)
c1.fit(X_train, y_train)
full_performance = (c1.predict(X_test) == y_test).mean()
print('Full Feature set performance: %.3f' % (full_performance))

c2 = RandomForestClassifier(max_depth=2, n_estimators=10, random_state=42)
c2.fit(X_train[:,pos==1], y_train)
subset_performance = (c2.predict(X_test[:,pos==1]) == y_test).mean()
print('Subset Feature set performance: %.3f' % (subset_performance))

2022-05-12 16:31:07,431 - pyswarms.discrete.binary - INFO - Optimize for 10 iters with {'c1': 1.9, 'c2': 1.9, 'w': 0.5, 'k': 10, 'p': 2}
pyswarms.discrete.binary: 100%|█████████████████████████|10/10, best_cost=0.0106
2022-05-12 16:31:32,540 - pyswarms.discrete.binary - INFO - Optimization finished | best cost: 0.010638297872340385, best pos: [0 0 1 1 0 0 1 0 1 0 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 0 0 1]


Optimiser ran for 25.1106 seconds
Optimiser retained 19 features
Full Feature set performance: 0.931
Subset Feature set performance: 0.941
